In [12]:
import torch
import math
from bayes_models import MLP
import bayes_utils as bu
prior_type = ["empirical", "wider_he", "wider_he"]

In [14]:
x = torch.randn(32, 1)
def base_model(x):
    return -(x+0.5)*torch.sin(3 * math.pi *x)

y = base_model(x)

In [3]:
model = MLP(1, 2, prior_type, hidden_dims=[128, 128])

In [5]:
pred = model(x)

In [9]:
log_variance = pred.mean[:, 1].reshape(-1)
mean = pred.mean[:, 0].reshape(-1)

sll = pred.var[:, 1, 1].reshape(-1)
smm = pred.var[:, 0, 0].reshape(-1)
sml = pred.var[:, 0, 1].reshape(-1)

In [11]:
def gaussian_loglikelihood_core(target, mean, log_variance, smm, sml, sll):
    return - 0.5 * (bu.log2pi + log_variance + torch.exp(-log_variance + 0.5 * sll) * (smm + (mean - sml - target)**2))

In [25]:
torch.sum(gaussian_loglikelihood_core(y, mean, log_variance, smm, sml, sll)).backward()

In [17]:
mean

tensor([-12.9549, -22.1776,  -9.5828, -12.9521, -43.5556, -15.2360, -11.2966,
        -10.3228,  -7.7707, -13.4948, -21.1982,  -8.8841,  -7.7061,  -9.9256,
        -44.7099, -35.6010, -19.0098, -24.7439, -14.5047,  -7.7389, -24.0988,
        -11.9629, -18.7767, -10.2708, -49.3751,  -8.0454, -16.1067, -16.0429,
        -28.1284,  -8.3606, -11.5523,  -8.6126], grad_fn=<AsStridedBackward>)

In [18]:
log_variance

tensor([-31.0155, -43.6555, -19.6397, -25.8772, -85.3174, -30.2376, -26.8893,
        -20.9691, -16.9639, -32.3527, -41.7547, -20.7677, -17.2200, -20.2507,
        -87.5711, -69.7938, -45.9281, -59.9666, -34.8486, -17.4348, -47.3878,
        -24.0095, -37.0636, -24.3143, -96.6814, -18.4841, -31.9107, -31.7880,
        -55.2283, -19.3742, -27.5277, -20.0529], grad_fn=<AsStridedBackward>)

In [20]:
y.reshape(-1)

tensor([ 0.4794, -1.2012, -1.5262,  0.0465,  0.1230,  0.1276,  0.2901,  0.1133,
        -0.5920, -0.2730,  0.3489, -0.1459,  0.3156, -1.3366, -0.2580,  0.2998,
        -1.8569, -2.4551, -0.0149,  2.2880, -0.0657,  0.9590,  0.5095, -0.5413,
         0.3386,  0.3358, -0.3903,  0.9257,  1.0322,  1.8084,  0.6478,  0.4367])

In [23]:
torch.exp(-log_variance)

tensor([2.9502e+13, 9.1064e+18, 3.3838e+08, 1.7311e+11, 1.1294e+37, 1.3552e+13,
        4.7630e+11, 1.2786e+09, 2.3298e+07, 1.1236e+14, 1.3610e+18, 1.0454e+09,
        3.0099e+07, 6.2338e+08, 1.0756e+38, 2.0467e+30, 8.8369e+19, 1.1045e+26,
        1.3631e+15, 3.7312e+07, 3.8040e+20, 2.6742e+10, 1.2489e+16, 3.6271e+10,
               inf, 1.0654e+08, 7.2217e+13, 6.3880e+13, 9.6685e+23, 2.5947e+08,
        9.0188e+11, 5.1151e+08], grad_fn=<ExpBackward>)

In [26]:
import torch.nn as nn

In [27]:
net = nn.Sequential(
    nn.Linear(1, 128),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Linear(128, 2),
    nn.ReLU()
)

In [28]:
net(x)

tensor([[0.2855, 0.0000],
        [0.4119, 0.0000],
        [0.4148, 0.0000],
        [0.1829, 0.0018],
        [0.1255, 0.0250],
        [0.1253, 0.0250],
        [0.0873, 0.0379],
        [0.2184, 0.0000],
        [0.4069, 0.0000],
        [0.2475, 0.0000],
        [0.1016, 0.0322],
        [0.1377, 0.0234],
        [0.1111, 0.0285],
        [0.2331, 0.0000],
        [0.1417, 0.0223],
        [0.0885, 0.0373],
        [0.3239, 0.0000],
        [0.4264, 0.0000],
        [0.0429, 0.0756],
        [0.3769, 0.0000],
        [0.0418, 0.0769],
        [0.2096, 0.0000],
        [0.3480, 0.0000],
        [0.1537, 0.0191],
        [0.0956, 0.0345],
        [0.0948, 0.0348],
        [0.3407, 0.0000],
        [0.2006, 0.0000],
        [0.3524, 0.0000],
        [0.3602, 0.0000],
        [0.0093, 0.0771],
        [0.2530, 0.0000]], grad_fn=<ThresholdBackward0>)